In [ ]:
%cd ..

In [ ]:
import glob

from datetime import datetime
import os
import polars as pl

logs = glob.glob("logs/benchmark-amip/*.log")

dates = []
sypds = []

for log in logs:
    date = datetime.fromisoformat(os.path.basename(log).split(".")[0])
    dates.append(date)
    with open(log, "r") as f:
        lines = f.readlines()
    sypd = None
    found_step_40 = False
    for line in lines:
        if "n_steps_completed = 40" in line:
            found_step_40 = True
        if found_step_40 and "estimated_sypd = " in line:
            sypd = float(
                line.split("estimated_sypd = ")[1].split()[0].replace('"', "")
            )
            break
    sypds.append(sypd)

df = pl.DataFrame({"date": dates, "sypd": sypds})
df = df.sort("date")

os.makedirs("results", exist_ok=True)

df.write_csv("results/benchmark-amip-sypd.csv")

df.to_pandas().plot(
    x="date", y="sypd", title="AMIP benchmark SYPD (16 horiz. elem.)"
)